In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
from glob import glob
from scipy.interpolate import interp1d
from scipy.ndimage.filters import gaussian_filter1d
from hapi import *
from joblib import Parallel, delayed

from matplotlib.font_manager import FontProperties

%run simulate_radiances_utils.py


font = {'weight' : 'bold',
        'size'   : 22}
label_fontdict = {'weight' : 'bold',
        'size'   : 22}
title_fontdict = {'weight' : 'bold',
        'size'   : 21}

matplotlib.rc('font', **font)

In [2]:
output_cs_matrix_rel_path = '/home/cchristo/proj_christian/data/cs_matrices/summit_parallel/'
output_cs_matrix_path_format = output_cs_matrix_rel_path + '{year}/{month:02d}/cs_matrix_{year}{month:02d}{day:02d}_{hour:02d}{minute:02d}.nc'

In [3]:
reanlysis_dir = '/home/cchristo/proj_christian/data/reanalysis_3d/merra2/2015/'
file_type = '*.nc4'

all_file_paths = [y for x in os.walk(reanlysis_dir) for y in glob(os.path.join(x[0], file_type))]

In [4]:
# all_file_paths
loc_lat =  72.5796
loc_lon = -38.4588

In [5]:
ds_3d = xr.open_mfdataset(all_file_paths, combine='by_coords')

In [30]:
# ds_single = ds_3d.sel(lat = loc_lat, lon = loc_lon , time = 0, method = 'nearest')
# ds_single.load()

In [6]:
%%capture
# Min wavenumber
xmin = 100
# Maximum wavenumber
xmax = 2100
# Actually downloading the data 
# (we have to know the HITRAN molecule numbers, given in http://hitran.org/docs/molec-meta/)
fetch('H2O_S',1,1,xmin,xmax)
fetch('CO2_S',2,1,xmin,xmax)
fetch('CH4_S',6,1,xmin,xmax)
# fetch('O2',7,1,xmin,xmax)
nu_,sw_ = getColumns('CO2_S',['nu','sw'])

In [7]:
# Let us hust get line position nu and line strength sw for the different molecules:
nu_H2O,sw_H2O = getColumns('H2O_S',['nu','sw'])
nu_CH4,sw_CH4 = getColumns('CH4_S',['nu','sw'])
nu_CO2,sw_CO2 = getColumns('CO2_S',['nu','sw'])

In [10]:
NLEV = len(ds_3d['lev'])

def make_cs_matrices_time_i(ds_3d, time_ii):
    '''
    
    Args
        time_ii - index of time to select
    '''
# # for time_i in ds_3d.time[:2]:
#     # Let us hust get line position nu and line strength sw for the different molecules:
#     try:
    xmin = 100
# Maximum wavenumber
    xmax = 2100
    # Actually downloading the data 
    # (we have to know the HITRAN molecule numbers, given in http://hitran.org/docs/molec-meta/)
    fetch('H2O_S',1,1,xmin,xmax)
    fetch('CO2_S',2,1,xmin,xmax)
    fetch('CH4_S',6,1,xmin,xmax)
    # fetch('O2',7,1,xmin,xmax)
    nu_,sw_ = getColumns('CO2_S',['nu','sw'])
    nu_H2O,sw_H2O = getColumns('H2O_S',['nu','sw'])
    nu_CH4,sw_CH4 = getColumns('CH4_S',['nu','sw'])
    nu_CO2,sw_CO2 = getColumns('CO2_S',['nu','sw'])
    print(time_i)
    ds_single = ds_3d.sel(lat = loc_lat, lon = loc_lon, method = 'nearest')
    ds_single = ds_single.isel(time = time_ii)
    time_i = ds_single.time
    ds_single.load()
    p_prof, T_prof, dz_prof,  vmr_h2o_prof, VCD_dry_prof, rho_N_h2o_prof, rho_N_prof = compute_profile_properties_merra2(ds_single, verbose=False)

    out_cs_matrix_path = output_cs_matrix_path_format.format(year = time_i['time.year'].item(), 
                                                             month = time_i['time.month'].item(), 
                                                             day = time_i['time.day'].item(),
                                                             hour = time_i['time.hour'].item(), 
                                                             minute = time_i['time.minute'].item())
    create_cross_section_matrix_hapi(p_prof, T_prof, 
                                 xmin, xmax, 
                                 output_path=out_cs_matrix_path)
#     except:
#         print('Could not process ', time_ii)

In [11]:

Parallel(n_jobs=40)(delayed(make_cs_matrices_time_i)(ds_3d, time_ii) for time_ii in range(len(ds_3d.time)))

Exception: PARSE ERROR: unknown format of the par value ()

In [29]:
create_cross_section_matrix_hapi(p_prof, T_prof, 
                                 xmin, xmax, 
                                 output_path=output_cs_matrix_rel_path + 'test.nc')

<xarray.Dataset>
Dimensions:        (nu: 200000, pressure: 72)
Coordinates:
  * nu             (nu) float64 100.0 100.0 100.0 ... 2.1e+03 2.1e+03 2.1e+03
  * pressure       (pressure) float32 1.5000001 2.6350012 ... 64638.945
Data variables:
    cs_matrix_co2  (nu, pressure) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cs_matrix_ch4  (nu, pressure) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cs_matrix_h2o  (nu, pressure) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [7]:
# nu_, cs_o2 = absorptionCoefficient_Voigt(SourceTables='O2', Environment={'p':1,'T':270}, WavenumberStep=1)

/home/cchristo/proj_christian/hapi.py:18953: UserWarning: Big wavenumber step: possible accuracy decline
  if OmegaStep>0.1: warn('Big wavenumber step: possible accuracy decline')


In [31]:
NLEV - 69

3

In [32]:
# nu_, cs_o2 = absorptionCoefficient_Voigt(SourceTables='O2', Environment={'p':1,'T':270}, WavenumberStep=1)

cs_matrix_co2 = np.zeros((len(nu_),NLEV))
cs_matrix_ch4 = np.zeros((len(nu_),NLEV))
cs_matrix_h2o = np.zeros((len(nu_),NLEV))
# Loop over each layer 
for i in range(NLEV - 69):
    print(str(i)+'/'+str(NLEV), end='\r')
    p_ = p_prof[i]/101325
#     print(p_)
    T_ = T_prof[i]
    nu_, cs_co2 = absorptionCoefficient_Voigt(SourceTables='CO2_S', WavenumberRange=[xmin,xmax],Environment={'p':p_,'T':T_},IntensityThreshold=1e-27)
    nu_, cs_ch4 = absorptionCoefficient_Voigt(SourceTables='CH4_S', WavenumberRange=[xmin,xmax],Environment={'p':p_,'T':T_},IntensityThreshold=1e-27)
    nu_, cs_h2o = absorptionCoefficient_Voigt(SourceTables='H2O_S', WavenumberRange=[xmin,xmax],Environment={'p':p_,'T':T_},IntensityThreshold=1e-27)
    cs_matrix_co2[:,i] = cs_co2
    cs_matrix_ch4[:,i] = cs_ch4
    cs_matrix_h2o[:,i] = cs_h2o